In [1]:
from keras.models import load_model
from collections import deque
import numpy as np
import argparse
import pickle
import cv2
import os

Using TensorFlow backend.


In [27]:
ap = argparse.ArgumentParser()
ap.add_argument("-i", "--input", required=True,
    help="path to our input video")
ap.add_argument("-o", "--output", required=True,
    help="path to our output video")
ap.add_argument("-s", "--size", type=int, default=128,
    help="size of queue for averaging")
args = vars(ap.parse_args())

usage: ipykernel_launcher.py [-h] -i INPUT -o OUTPUT [-s SIZE]
ipykernel_launcher.py: error: the following arguments are required: -i/--input, -o/--output


SystemExit: 2

In [2]:
path = os.getcwd()
model =  load_model(os.path.join(path, "Model/model2.h5"))
lb = pickle.loads(open ("lb.pickle", "rb").read())

W0904 09:15:26.278086 139690559530816 deprecation_wrapper.py:119] From /home/ligirk/anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0904 09:15:26.304141 139690559530816 deprecation_wrapper.py:119] From /home/ligirk/anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:4185: The name tf.truncated_normal is deprecated. Please use tf.random.truncated_normal instead.

W0904 09:15:26.321133 139690559530816 deprecation_wrapper.py:119] From /home/ligirk/anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:245: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0904 09:15:26.321664 139690559530816 deprecation_wrapper.py:119] From /home/ligirk/anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:174: The name tf.get_default_session is deprecated. Please use tf.compat.v1.ge

In [3]:
lb.classes_

array(['.ipynb_checkpoints', 'badminton', 'baseball', 'basketball',
       'boxing', 'chess', 'cricket', 'fencing', 'football', 'formula1',
       'gymnastics', 'hockey', 'ice_hockey', 'kabaddi', 'motogp',
       'shooting', 'swimming', 'table_tennis', 'tennis', 'volleyball',
       'weight_lifting', 'wrestling', 'wwe'], dtype='<U18')

In [30]:
mean = np.array([123.68, 116.779, 103.939], dtype="float32")
qu = deque(maxlen=args["size"])
vid = cv2.VideoCapture(os.path.join(path, "Input", args["input"]))
writer = None
(W, H) = (None, None)

NameError: name 'args' is not defined

In [11]:
while True:
    grabbed, frame = vid.read()
    if not grabbed:
        break;
    if H is None or W is None:
        (W, H) = frame.shape[:2]
    output = frame.copy()
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    frame = cv2.resize(frame, (224, 224)).astype("float32")
    frame -= mean
    frame = np.reshape(frame, (1, 224, 224, 3))
    preds = model.predict(frame)
    qu.append(preds)
    result = np.array(qu).mean(axis = 0)
    index = np.argmax(result)
    label = lb.classes_[index]
    string = "The activity is: {}".format(label)
    cv2.putText(output, string, (35, 50), cv2.FONT_HERSHEY_SIMPLEX,
        1.25, (50, 205, 154), 5)
    if writer is None:
        fourcc = cv2.VideoWriter_fourcc(*"MJPG")
        writer = cv2.VideoWriter(os.path.join(path, "Output", args["output"]), fourcc, 30, (W, H), True)
    writer.write(output)
    cv2.imshow("Output", output)
    key = cv2.waitKey(1) & 0xFF
 
    if key == ord("q"):
        break
        
writer.release()
vid.release()
cv2.destroyAllWindows()

NameError: name 'cv2' is not defined